# Install Necessary Libraries

In [ ]:
%%capture
!pip install emoji
!pip install translators

# Import Libraries

In [ ]:
import re
import pandas as pd
import emoji
import translators as ts

Using region Groningen server backend.



# Load the Dataset

In [ ]:
dataset_file_location = '/content/drive/MyDrive/B.Tech. Final Year Project/Offensive Language Detection/Datasets/m_dataset_18_9/merged_dataset_18_9_arnab.csv'
df = pd.read_csv(dataset_file_location)
df.count()

text    38530
hate    38530
dtype: int64

In [ ]:
df['hate'].value_counts()

0    19588
1    18942
Name: hate, dtype: int64

# Data Pre-processing

## Load Bengali Numbers

In [ ]:
with open('/content/drive/MyDrive/B.Tech. Final Year Project/Offensive Language Detection/Preprocess Files/numbers_bn.txt', 'r', encoding='utf-8') as f:
    num_bn = f.read()
    num_bn = [char for char in num_bn]
print(num_bn)

['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']


## Load Bengali Punctuations

In [ ]:
with open('/content/drive/MyDrive/B.Tech. Final Year Project/Offensive Language Detection/Preprocess Files/own_punctuations-bn.txt', 'r', encoding='utf-8') as f:
    punctuations = f.read()
    punctuations = [char for char in punctuations]
print('Number of Punctuations: ', len(punctuations))
print(punctuations)

Number of Punctuations:  37
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '।', '‘', '’', '“', '”']


## Remove Censored Comments
Some comments have multiple stars(```*```) to hide abusive words. Those comments are removed.

In [ ]:
df_clean = df[~df['text'].str.contains(r'\*{2,}', regex=True)].copy()

## Remove Special Words

In [ ]:
special_words = ['<br />', ':D', ':P', ':v']

def remove_special_wrods(string):
    for word in special_words:
        string = string.replace(word, ' ')
    return string

In [ ]:
df_clean['text'] = df_clean['text'].apply(remove_special_wrods)
df_clean.head()

,text,hate
0,.... ঐ ইন্দুর তোই মরছ নাই?,1
1,✈✈✈✈ মুরগি চোরের পাছায় ডুকবি আর মারবি।।।,1
2,২য় মীর জাফরের মুখে মুতে দে...,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


## Remove Emoji

In [ ]:
df_clean['text'] = df_clean['text'].apply(lambda x: ''.join([char for char in x if not emoji.emoji_count(char)]))
df_clean.head()

,text,hate
0,.... ঐ ইন্দুর তোই মরছ নাই?,1
1,মুরগি চোরের পাছায় ডুকবি আর মারবি।।।,1
2,২য় মীর জাফরের মুখে মুতে দে...,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


## Remove Punctuations

In [ ]:
df_clean['text'] = df_clean['text'].apply(lambda x: ''.join([char for char in x if char not in punctuations]))
df_clean.head()

,text,hate
0,ঐ ইন্দুর তোই মরছ নাই,1
1,মুরগি চোরের পাছায় ডুকবি আর মারবি,1
2,২য় মীর জাফরের মুখে মুতে দে,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


## Remove Numbers

### Remove English Numbers

In [ ]:
df_clean['text'] = df_clean['text'].apply(lambda x: ''.join([char for char in x if not char.isdigit()]))
df_clean.head()

,text,hate
0,ঐ ইন্দুর তোই মরছ নাই,1
1,মুরগি চোরের পাছায় ডুকবি আর মারবি,1
2,য় মীর জাফরের মুখে মুতে দে,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


### Remove Bengali Numbers

In [ ]:
df_clean['text'] = df_clean['text'].apply(lambda x: ''.join([char for char in x if char not in num_bn]))
df_clean.head()

,text,hate
0,ঐ ইন্দুর তোই মরছ নাই,1
1,মুরগি চোরের পাছায় ডুকবি আর মারবি,1
2,য় মীর জাফরের মুখে মুতে দে,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


## Remove Single-Letter Words

In [ ]:
df_clean['text'] = df_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
df_clean.head()

,text,hate
0,ইন্দুর তোই মরছ নাই,1
1,মুরগি চোরের পাছায় ডুকবি আর মারবি,1
2,মীর জাফরের মুখে মুতে দে,1
3,অই জারছি পরা লোক টা কে পাইলে কোত্তার মতো পিটাইতাম,1
4,অকে পিথিবির আলো দেখানোর আগেই মেরে ফেলা দরকার ছিল,1


## Translate English words to Bengali

If the below code gives an error like:<br />
**```KeyError: 0```**<br />

Then run the code until and unless the error is gone.<br />
This can happen due to some problem in the translator.

> Translator Model Used: **Alibaba**


In [ ]:
for row in df_clean.itertuples():
    temp_text = row.text
    for word in temp_text.split():
        if re.match(r'^[a-zA-Z]+$', word):
            word_to_bn = ts.translate_text(word, from_language='en', to_language='bn')
            if word_to_bn == word:
                word_to_bn = ''
            temp_text = temp_text.replace(word, word_to_bn)
            df_clean.loc[row.Index, 'text'] = temp_text

## Remove Blank Comments

In [ ]:
df_clean.drop(df_clean[df_clean['text'].str.len() == 0].index, axis=0, inplace=True)
df_clean.reset_index(drop=True, inplace=True)

# Cleaned Dataset Details

In [ ]:
df_clean.count()

text    38353
hate    38353
dtype: int64

In [ ]:
print(df.count()['text'] - df_clean.count()['text'], "rows were removed after pre-processing.")

177 rows were removed after pre-processing.


In [ ]:
print("Maximum string length:", df_clean['text'].str.len().max())
print("Minimum string length:", df_clean['text'].str.len().min())

Maximum string length: 49
Minimum string length: 2


# Saving the Cleaned Dataset

In [ ]:
df_clean_name = 'cleaned_'+dataset_file_location.rsplit('/', 1)[-1].rsplit('.')[0]
df_clean.to_csv('/content/drive/MyDrive/B.Tech. Final Year Project/Offensive Language Detection/Datasets/'+df_clean_name+'.csv', index=False)